<a href="https://colab.research.google.com/github/PaoloComensoli/MasterDegree/blob/main/Decision%20Models/Assignments/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

In [ ]:
from pulp import *
import pandas as pd

In [ ]:
model = LpProblem("MaximizeProfit", LpMaximize)

In [ ]:
# Variables
cargos = ['Cargo 1', 'Cargo 2', 'Cargo 3']
wagons = ['Wagon 1','Wagon 2','Wagon 3','Wagon 4']
key = [(c,w) for c in cargos for w in wagons]
transport = LpVariable.dicts('TRANSPORT', key, lowBound=0, cat='Continuous')

# Variables for objective functions
revenues = {
    'Cargo 1' : 3500,
    'Cargo 2' : 2500,
    'Cargo 3' : 2000
}

# Variables for constraints
cargo_availability = {
    'Cargo 1' : 20,
    'Cargo 2' : 10,
    'Cargo 3' : 18
}

cargo_space = {
    'Cargo 1' : 500,
    'Cargo 2' : 300,
    'Cargo 3' : 400
}

wagons_availability = {
    'Wagon 1' : (10, 5000),
    'Wagon 2' : (8, 4000),
    'Wagon 3' : (12, 8000),
    'Wagon 4' : (6, 2500),
}

In [ ]:
# Define Objective
model += lpSum([revenues[k[0]] * transport[k] for k in key])

In [ ]:
# Define Constraints
for c in cargos:
    model += lpSum([transport[k] for k in key if k[0] == c]) <= cargo_availability[c]
    
for w in wagons:
    model += lpSum([transport[k] for k in key if k[1] == w]) <= wagons_availability[w][0]

for w in wagons:
    model += lpSum([cargo_space[k[0]] * transport[k] for k in key if w[1] == w]) <= wagons_availability[w][1]

In [1]:
model.writeLP("assignment1.lp") # Store the problem
model.solve(PULP_CBC_CMD(msg=False)) # Keep the output clean
status = LpStatus[model.status]
print(status)

In [1]:
key_values = {v: k for k, v in transport.items()}
print('Optimal value:', model.objective.value(), 'of revenue')
print('')
print('Output variables: ')

current_cargo = ''
for v in model.variables():
    cargo = key_values[v][0]
    if cargo != current_cargo:
        print('******* ' + cargo + ' *******')
        current_cargo = cargo
    print(v.varValue, 'tons of', cargo, 'on', key_values[v][1])

In [1]:
# Sensitivity - Shadow Price & Slack <- only for the old version
s = [{'Contraint Name': name, 'Shadow Price': c.pi, 'Slack': c.slack}
     for name, c in model.constraints.items()]
print(pd.DataFrame(s))

# Sensitivity Analysis - GLPK
Prerequisites: The solver GLPK must be installed

In [ ]:
model.solve(GLPK(msg=True, options=['--ranges', 'assignment1_Sensitivity.sen'])) # Solve and produce the sensitivity report